# Microsoft AutoGen: Zero to Hero Guide

## Building Conversational Multi-Agent Systems

**Objective:** This comprehensive notebook takes you from beginner to advanced AutoGen user. You'll learn how to build single agents, multi-agent teams, and complex conversational workflows.

**Target Audience:** Software engineers from complete beginners to experts looking to master AutoGen.

---

## Table of Contents
1. [Introduction & Core Philosophy](#1-introduction--core-philosophy)
2. [Prerequisites & Setup](#2-prerequisites--setup)
3. [Core Concepts Deep Dive](#3-core-concepts-deep-dive)
4. [Your First Agent: Basic Two-Agent Chat](#4-your-first-agent-basic-two-agent-chat)
5. [Understanding Agent Configuration](#5-understanding-agent-configuration)
6. [Custom System Messages & Personas](#6-custom-system-messages--personas)
7. [Human-in-the-Loop Patterns](#7-human-in-the-loop-patterns)
8. [Multi-Agent Group Chats](#8-multi-agent-group-chats)
9. [Custom Termination Conditions](#9-custom-termination-conditions)
10. [Function Calling & Tools](#10-function-calling--tools)
11. [Nested Chats & Sequential Workflows](#11-nested-chats--sequential-workflows)
12. [Best Practices & Common Pitfalls](#12-best-practices--common-pitfalls)
13. [Conclusion & Next Steps](#13-conclusion--next-steps)

---

## 1. Introduction & Core Philosophy

### What is AutoGen?

**AutoGen** is Microsoft's open-source framework for building multi-agent conversational AI systems. Unlike single-agent frameworks, AutoGen's core insight is that **complex tasks can be solved by a team of specialized agents having conversations**.

### Core Philosophy

| Principle | Description |
|-----------|-------------|
| **Conversational** | Agents solve problems through dialogue, not isolated actions |
| **Collaborative** | Multiple agents with different roles work together |
| **Flexible** | Supports human participation, code execution, and tool use |
| **Composable** | Build complex systems from simple, reusable agent patterns |

### When to Use AutoGen?

✅ **Good for:**
- Multi-agent problem solving (coding, research, planning)
- Tasks requiring iterative refinement through conversation
- Human-in-the-loop workflows
- Code generation and execution pipelines

❌ **Consider alternatives when:**
- You need simple single-agent tools (use LangChain)
- You need complex state machines (use LangGraph)
- You need role-playing crews with specific processes (use CrewAI)

---

## 2. Prerequisites & Setup

### Requirements

Before starting, ensure you have:

- **Python 3.8+** (3.10+ recommended)
- **OpenAI API Key** (or compatible API)
- Basic understanding of Python and LLMs

### Installation

```bash
# Install AutoGen with all dependencies
pip install pyautogen

# For additional features (optional)
pip install pyautogen[teachable]  # For teachable agents
pip install pyautogen[redis]      # For Redis-backed memory
```

In [ ]:
# Install dependencies (uncomment to run)
# !pip install pyautogen python-dotenv

In [ ]:
import os
import warnings
from dotenv import load_dotenv

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Load environment variables from .env file
load_dotenv()

# Verify API key is available
if not os.getenv("OPENAI_API_KEY"):
    print("❌ ERROR: OPENAI_API_KEY not found.")
    print("Please create a .env file with: OPENAI_API_KEY=your_key_here")
else:
    print("✅ API Key loaded successfully!")

---

## 3. Core Concepts Deep Dive

Before writing code, let's understand AutoGen's building blocks:

### Agent Types

```
┌─────────────────────────────────────────────────────────────────┐
│                        AutoGen Agents                          │
├─────────────────────────┬───────────────────────────────────────┤
│   AssistantAgent        │   LLM-powered agent that generates    │
│                         │   responses, writes code, reasons     │
├─────────────────────────┼───────────────────────────────────────┤
│   UserProxyAgent        │   Proxy for human/automated actions   │
│                         │   Can execute code and provide input  │
├─────────────────────────┼───────────────────────────────────────┤
│   ConversableAgent      │   Base class for all agents           │
│                         │   Maximum customization flexibility   │
├─────────────────────────┼───────────────────────────────────────┤
│   GroupChatManager      │   Orchestrates multi-agent group      │
│                         │   conversations and turn-taking       │
└─────────────────────────┴───────────────────────────────────────┘
```

### The Conversation Loop

```
User/Initiator
      │
      ▼
┌─────────────┐     message      ┌─────────────┐
│   Agent A   │ ──────────────▶  │   Agent B   │
│ (UserProxy) │                  │ (Assistant) │
└─────────────┘  ◀──────────────  └─────────────┘
      │              response           │
      │                                 │
      └── Execute Code ◀────────────────┘
              │
              ▼
         Observation
              │
              ▼
      Continue or TERMINATE
```

### Key Configuration Parameters

| Parameter | Description | Default |
|-----------|-------------|--------|
| `llm_config` | LLM model and API configuration | Required |
| `system_message` | Agent's persona and instructions | Built-in default |
| `human_input_mode` | When to ask for human input | "ALWAYS" |
| `max_consecutive_auto_reply` | Auto-reply limit before stopping | 100 |
| `code_execution_config` | How/where to run code | None |
| `is_termination_msg` | Function to detect end of conversation | None |

---

## 4. Your First Agent: Basic Two-Agent Chat

Let's start with the simplest AutoGen pattern: a two-agent system where one agent writes code and another executes it.

### The Pattern
1. **UserProxyAgent** - Acts as the user, can execute code
2. **AssistantAgent** - LLM-powered, writes responses/code

In [ ]:
import autogen

# Step 1: Configure the LLM
# AutoGen uses a 'config_list' format for flexibility (supports multiple models/fallbacks)
config_list = [
    {
        'model': 'gpt-4o',
        'api_key': os.getenv("OPENAI_API_KEY"),
    }
]

# LLM configuration with additional parameters
llm_config = {
    "config_list": config_list,
    "temperature": 0,  # More deterministic responses
    "timeout": 120,    # Timeout in seconds
}

print("✅ LLM Configuration ready!")
print(f"   Model: {config_list[0]['model']}")

In [ ]:
# Step 2: Create the Assistant Agent
# This agent uses the LLM to generate responses

assistant = autogen.AssistantAgent(
    name="Assistant",
    llm_config=llm_config,
    # Default system message is good for general coding tasks
)

print("✅ Assistant Agent created!")
print(f"   Name: {assistant.name}")
print(f"   Type: AssistantAgent (LLM-powered)")

In [ ]:
# Step 3: Create the User Proxy Agent
# This agent acts as the user and can execute code

user_proxy = autogen.UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",  # Fully automated (no human input required)
    max_consecutive_auto_reply=5,  # Limit conversation turns
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "autogen_workspace",  # Directory for code execution
        "use_docker": False,  # Set True for sandboxed execution
    },
)

print("✅ User Proxy Agent created!")
print(f"   Name: {user_proxy.name}")
print(f"   Human Input Mode: NEVER (fully automated)")
print(f"   Code Execution: Enabled in 'autogen_workspace/'")

In [ ]:
# Step 4: Start the conversation!

print("\n" + "="*60)
print("🚀 STARTING TWO-AGENT CONVERSATION")
print("="*60 + "\n")

# The user_proxy initiates the chat with a task
result = user_proxy.initiate_chat(
    assistant,
    message="What is today's date? Write Python code to find and print it.",
)

print("\n" + "="*60)
print("✅ CONVERSATION COMPLETE")
print("="*60)

In [ ]:
# Step 5: Analyze the conversation history

print("\n📊 CONVERSATION ANALYSIS")
print("-" * 40)
print(f"Total messages: {len(result.chat_history)}")
print(f"Summary: {result.summary[:200]}..." if len(result.summary) > 200 else f"Summary: {result.summary}")
print(f"Cost: {result.cost}")

### 🎯 Key Takeaways from Basic Example

1. **AssistantAgent** generates code when asked
2. **UserProxyAgent** automatically executes the code
3. The conversation continues until `TERMINATE` is detected
4. All code runs in the specified `work_dir`

---

## 5. Understanding Agent Configuration

Let's explore the key configuration options in detail.

In [ ]:
# Demonstration of different LLM configurations

# Configuration 1: Basic OpenAI
config_basic = {
    "config_list": [{"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")}],
}

# Configuration 2: With fallback models
config_with_fallback = {
    "config_list": [
        {"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")},
        {"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY")},  # Fallback
    ],
    "timeout": 60,
}

# Configuration 3: With caching (reduces API calls during development)
config_with_cache = {
    "config_list": [{"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")}],
    "cache_seed": 42,  # Same seed = same cached responses
}

print("✅ Configuration examples ready!")
print("   - Basic: Single model")
print("   - Fallback: Primary + backup model")
print("   - Cached: Repeatable results during development")

In [ ]:
# Demonstration of Human Input Modes

# Mode 1: NEVER - Fully automated
auto_proxy = autogen.UserProxyAgent(
    name="AutoProxy",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "auto_workspace", "use_docker": False},
)

# Mode 2: ALWAYS - Human approves every response
interactive_proxy = autogen.UserProxyAgent(
    name="InteractiveProxy",
    human_input_mode="ALWAYS",  # Will prompt for input
    code_execution_config={"work_dir": "interactive_workspace", "use_docker": False},
)

# Mode 3: TERMINATE - Human input only at termination
semi_auto_proxy = autogen.UserProxyAgent(
    name="SemiAutoProxy",
    human_input_mode="TERMINATE",  # Auto until termination signal
    code_execution_config={"work_dir": "semiauto_workspace", "use_docker": False},
)

print("\n📋 HUMAN INPUT MODES")
print("-" * 40)
print("NEVER:     Fully automated, no human interaction")
print("ALWAYS:    Human approval required for each turn")
print("TERMINATE: Automated until conversation ends")

---

## 6. Custom System Messages & Personas

The `system_message` defines your agent's personality, expertise, and behavior. This is crucial for specialized agents.

In [ ]:
# Example 1: Python Expert Agent
python_expert = autogen.AssistantAgent(
    name="PythonExpert",
    system_message="""You are a senior Python developer with 15 years of experience.
    
Your expertise includes:
- Writing clean, PEP-8 compliant code
- Performance optimization
- Design patterns and best practices
- Testing and debugging

When writing code:
1. Always include type hints
2. Add comprehensive docstrings
3. Handle edge cases and errors
4. Suggest improvements when appropriate

End your response with TERMINATE when the task is complete.""",
    llm_config=llm_config,
)

print("✅ Python Expert Agent created!")

In [ ]:
# Example 2: Code Reviewer Agent
code_reviewer = autogen.AssistantAgent(
    name="CodeReviewer",
    system_message="""You are a meticulous code reviewer focused on quality and security.

When reviewing code, evaluate:
1. **Correctness**: Does the code do what it's supposed to?
2. **Security**: Are there potential vulnerabilities?
3. **Performance**: Can it be optimized?
4. **Readability**: Is the code clean and maintainable?
5. **Best Practices**: Does it follow conventions?

Provide feedback in this format:
- ✅ APPROVED: If code meets all criteria
- 🔄 REVISION NEEDED: If improvements are required (list them)
- ❌ REJECTED: If there are critical issues

Be constructive and specific in your feedback.""",
    llm_config=llm_config,
)

print("✅ Code Reviewer Agent created!")

In [ ]:
# Test the Python Expert
print("\n" + "="*60)
print("🐍 PYTHON EXPERT DEMONSTRATION")
print("="*60 + "\n")

result = user_proxy.initiate_chat(
    python_expert,
    message="Write a function to calculate the Fibonacci sequence up to n terms with memoization.",
)

---

## 7. Human-in-the-Loop Patterns

Real-world applications often need human oversight. AutoGen provides several patterns for this.

In [ ]:
# Pattern 1: Human Approval for Sensitive Operations

human_admin = autogen.UserProxyAgent(
    name="HumanAdmin",
    system_message="A human administrator who approves operations.",
    human_input_mode="ALWAYS",  # Always ask for human input
    max_consecutive_auto_reply=0,  # Never auto-reply
    code_execution_config=False,  # No code execution
)

print("✅ Human Admin Agent created!")
print("   This agent always waits for human input before proceeding.")
print("   Use for: Approvals, sensitive decisions, quality gates")

In [ ]:
# Pattern 2: Human Override with Auto-Continue

def should_request_human_input(message):
    """Custom logic to determine when human input is needed."""
    keywords = ["delete", "deploy", "production", "database", "critical"]
    content = message.get("content", "").lower()
    return any(keyword in content for keyword in keywords)

# Note: This is a conceptual example - actual implementation may vary
print("✅ Custom human input logic defined!")
print("   Triggers on keywords: delete, deploy, production, database, critical")

---

## 8. Multi-Agent Group Chats

AutoGen's `GroupChat` enables multiple agents to collaborate on complex tasks. This is where AutoGen truly shines!

In [ ]:
# Create a development team with specialized roles

# Product Manager
product_manager = autogen.AssistantAgent(
    name="ProductManager",
    system_message="""You are a Product Manager who breaks down requirements.
    
Your responsibilities:
- Clarify user requirements
- Define acceptance criteria
- Prioritize features
- Ensure the team understands the goal

Keep your responses concise and actionable.""",
    llm_config=llm_config,
)

# Developer
developer = autogen.AssistantAgent(
    name="Developer",
    system_message="""You are a Senior Developer who writes clean, efficient code.
    
Your responsibilities:
- Write production-quality code
- Follow best practices and design patterns
- Consider edge cases and error handling
- Document your code

Always provide working code, not pseudocode.""",
    llm_config=llm_config,
)

# QA Engineer
qa_engineer = autogen.AssistantAgent(
    name="QAEngineer",
    system_message="""You are a QA Engineer who ensures code quality.
    
Your responsibilities:
- Review code for bugs and issues
- Write test cases
- Verify edge cases are handled
- Approve code with 'APPROVED' or request changes

Be thorough but constructive in your feedback.""",
    llm_config=llm_config,
)

# User Proxy (executes code and represents the user)
executor = autogen.UserProxyAgent(
    name="Executor",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    is_termination_msg=lambda x: "APPROVED" in x.get("content", ""),
    code_execution_config={"work_dir": "team_workspace", "use_docker": False},
)

print("✅ Development Team created!")
print("   - ProductManager: Requirements & planning")
print("   - Developer: Code implementation")
print("   - QAEngineer: Review & testing")
print("   - Executor: Code execution")

In [ ]:
# Set up the Group Chat

groupchat = autogen.GroupChat(
    agents=[executor, product_manager, developer, qa_engineer],
    messages=[],
    max_round=12,  # Maximum conversation rounds
    speaker_selection_method="auto",  # LLM decides who speaks next
)

# The manager orchestrates the conversation
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

print("✅ Group Chat configured!")
print(f"   Max rounds: {groupchat.max_round}")
print(f"   Speaker selection: {groupchat.speaker_selection_method}")

In [ ]:
# Run the Group Chat!

print("\n" + "="*60)
print("👥 DEVELOPMENT TEAM GROUP CHAT")
print("="*60 + "\n")

task = """
Build a Python function that validates email addresses.
Requirements:
- Check for valid email format
- Handle common edge cases
- Return True/False with optional error message
"""

result = executor.initiate_chat(
    manager,
    message=task,
)

print("\n" + "="*60)
print("✅ GROUP CHAT COMPLETE")
print("="*60)

---

## 9. Custom Termination Conditions

Control when conversations end with custom termination logic.

In [ ]:
# Example termination conditions

# 1. Simple keyword termination
def terminate_on_keyword(msg):
    content = msg.get("content", "")
    return content.rstrip().endswith("TERMINATE")

# 2. Multiple termination keywords
def terminate_on_multiple_keywords(msg):
    content = msg.get("content", "").upper()
    keywords = ["TERMINATE", "DONE", "COMPLETE", "APPROVED"]
    return any(keyword in content for keyword in keywords)

# 3. Conditional termination (e.g., after successful execution)
def terminate_on_success(msg):
    content = msg.get("content", "")
    # Check for successful code execution
    if "exitcode: 0" in content:
        return True
    if "TERMINATE" in content:
        return True
    return False

# 4. Max cost termination (prevent runaway costs)
class CostTracker:
    def __init__(self, max_cost=1.0):
        self.total_cost = 0
        self.max_cost = max_cost
    
    def check_and_update(self, msg):
        # This is simplified - actual implementation would track API costs
        self.total_cost += 0.01  # Estimate per message
        if self.total_cost >= self.max_cost:
            return True
        return "TERMINATE" in msg.get("content", "")

print("✅ Termination condition examples defined!")

In [ ]:
# Demonstrate custom termination

smart_proxy = autogen.UserProxyAgent(
    name="SmartProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=terminate_on_success,  # Custom termination
    code_execution_config={"work_dir": "smart_workspace", "use_docker": False},
)

print("\n" + "="*60)
print("🎯 CUSTOM TERMINATION DEMO")
print("="*60 + "\n")

result = smart_proxy.initiate_chat(
    assistant,
    message="Print 'Hello World' in Python. The code should run successfully.",
)

---

## 10. Function Calling & Tools

AutoGen agents can call custom Python functions as tools. This extends their capabilities beyond text generation.

In [ ]:
from typing import Annotated, Literal

# Define custom tools/functions

def get_weather(
    city: Annotated[str, "The city to get weather for"],
) -> str:
    """Get the current weather for a city."""
    # Simulated weather data
    weather_data = {
        "new york": "72°F, Partly Cloudy",
        "london": "59°F, Rainy",
        "tokyo": "68°F, Sunny",
        "sydney": "77°F, Clear",
    }
    city_lower = city.lower()
    return weather_data.get(city_lower, f"Weather data not available for {city}")

def calculate(
    operation: Annotated[Literal["add", "subtract", "multiply", "divide"], "The math operation"],
    a: Annotated[float, "First number"],
    b: Annotated[float, "Second number"],
) -> str:
    """Perform a mathematical calculation."""
    if operation == "add":
        return str(a + b)
    elif operation == "subtract":
        return str(a - b)
    elif operation == "multiply":
        return str(a * b)
    elif operation == "divide":
        if b == 0:
            return "Error: Division by zero"
        return str(a / b)

print("✅ Custom tools defined: get_weather, calculate")

In [ ]:
# Create an agent with function calling capabilities

tool_assistant = autogen.AssistantAgent(
    name="ToolAssistant",
    system_message="""You are a helpful assistant with access to weather and calculator tools.
    Use the tools when appropriate to answer user questions.
    Reply TERMINATE when the task is complete.""",
    llm_config=llm_config,
)

tool_user = autogen.UserProxyAgent(
    name="ToolUser",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", ""),
    code_execution_config=False,  # No code execution, just function calls
)

# Register the functions with both agents
# The assistant decides when to call them, the user executes them
tool_assistant.register_for_llm(name="get_weather", description="Get weather for a city")(get_weather)
tool_assistant.register_for_llm(name="calculate", description="Perform math calculations")(calculate)

tool_user.register_for_execution(name="get_weather")(get_weather)
tool_user.register_for_execution(name="calculate")(calculate)

print("✅ Tool-enabled agents created!")

In [ ]:
# Test function calling

print("\n" + "="*60)
print("🔧 FUNCTION CALLING DEMO")
print("="*60 + "\n")

result = tool_user.initiate_chat(
    tool_assistant,
    message="What's the weather in Tokyo? Also, what is 25 multiplied by 4?",
)

---

## 11. Nested Chats & Sequential Workflows

For complex workflows, you can chain conversations or nest them within each other.

In [ ]:
# Sequential Workflow: Research -> Write -> Review

researcher = autogen.AssistantAgent(
    name="Researcher",
    system_message="""You are a researcher who gathers information.
    Summarize key points concisely. End with 'RESEARCH COMPLETE'.""",
    llm_config=llm_config,
)

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="""You are a technical writer.
    Take research findings and create clear documentation.
    End with 'WRITING COMPLETE'.""",
    llm_config=llm_config,
)

editor = autogen.AssistantAgent(
    name="Editor",
    system_message="""You are an editor who reviews and improves content.
    Provide the final polished version. End with 'TERMINATE'.""",
    llm_config=llm_config,
)

print("✅ Sequential workflow agents created!")

In [ ]:
# Run sequential workflow manually

coordinator = autogen.UserProxyAgent(
    name="Coordinator",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config=False,
)

print("\n" + "="*60)
print("📝 SEQUENTIAL WORKFLOW: Research -> Write -> Edit")
print("="*60)

# Step 1: Research
print("\n--- STEP 1: RESEARCH ---")
research_result = coordinator.initiate_chat(
    researcher,
    message="Research the benefits of using multi-agent AI systems in software development.",
    max_turns=2,
)
research_summary = research_result.chat_history[-1]["content"]

# Step 2: Write
print("\n--- STEP 2: WRITE ---")
write_result = coordinator.initiate_chat(
    writer,
    message=f"Based on this research, write a brief technical overview:\n\n{research_summary}",
    max_turns=2,
)
written_content = write_result.chat_history[-1]["content"]

# Step 3: Edit
print("\n--- STEP 3: EDIT ---")
edit_result = coordinator.initiate_chat(
    editor,
    message=f"Please review and polish this content:\n\n{written_content}",
    max_turns=2,
)

print("\n" + "="*60)
print("✅ SEQUENTIAL WORKFLOW COMPLETE")
print("="*60)

---

## 12. Best Practices & Common Pitfalls

### ✅ Best Practices

1. **Clear System Messages**: Be specific about roles, capabilities, and expected behavior
2. **Appropriate Termination**: Always define clear termination conditions
3. **Cost Management**: Use caching during development, set max_consecutive_auto_reply
4. **Security**: Use Docker for code execution in production
5. **Logging**: Enable verbose mode for debugging, disable in production

### ❌ Common Pitfalls

1. **Infinite Loops**: Forgetting termination conditions
2. **Vague Roles**: Agents don't know when to act without clear system messages
3. **Unsandboxed Execution**: Running untrusted code without Docker
4. **No Fallbacks**: Not handling API errors or timeouts
5. **Over-complexity**: Starting with too many agents instead of iterating

In [ ]:
# Production-ready configuration template

production_llm_config = {
    "config_list": [
        {"model": "gpt-4o", "api_key": os.getenv("OPENAI_API_KEY")},
        {"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY")},  # Fallback
    ],
    "temperature": 0,
    "timeout": 120,
    # "cache_seed": None,  # Disable caching in production
}

production_user_proxy = autogen.UserProxyAgent(
    name="ProductionProxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=terminate_on_multiple_keywords,
    code_execution_config={
        "work_dir": "production_workspace",
        "use_docker": True,  # Sandboxed execution
        "timeout": 60,
        "last_n_messages": 3,
    },
)

print("✅ Production-ready configuration template created!")

---

## 13. Conclusion & Next Steps

### What You've Learned

| Topic | Key Takeaway |
|-------|-------------|
| Core Concepts | AutoGen enables conversational multi-agent systems |
| Agent Types | AssistantAgent (LLM) + UserProxyAgent (execution) |
| Configuration | llm_config, system_message, human_input_mode |
| Group Chats | Multiple agents collaborating with GroupChatManager |
| Function Calling | Extend agents with custom Python tools |
| Workflows | Sequential and nested conversation patterns |

### Next Steps

1. **Practice**: Build a multi-agent system for your use case
2. **Explore**: Try AutoGen Studio for visual agent building
3. **Compare**: See how LangGraph handles similar workflows differently
4. **Production**: Implement proper error handling and monitoring

### Resources

- [AutoGen Documentation](https://microsoft.github.io/autogen/)
- [AutoGen GitHub](https://github.com/microsoft/autogen)
- [AutoGen Studio](https://microsoft.github.io/autogen/docs/autogen-studio/getting-started)

---

**Congratulations!** You've completed the AutoGen Zero to Hero guide! 🎉